# BertForMaskedLM을 이용한 [MASK] 예측
- KLUE 데이터를 기반으로 훈련된 BERT 모델을 허깅페이스에서 다운 받고, 토큰을 가린 문장에 대해 잘 예측하는지 확인

In [25]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

model_name = "klue/bert-base"

In [26]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [27]:
text = "[CLS] 대한민국의 수도는 서울입니다. [SEP] 서울의 대표적인 관광지는 남산타워 입니다. [SEP]"
tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)

['[CLS]', '대한민국', '##의', '수도', '##는', '서울', '##입니다', '.', '[SEP]', '서울', '##의', '대표', '##적인', '관광지', '##는', '남산', '##타', '##워', '입니다', '.', '[SEP]']


In [28]:
# 가릴 부분 지정
masked_index = 15
print(tokenized_text[masked_index])

남산


In [29]:
# 마스크 적용
tokenized_text[masked_index] = "[MASK]"
print(tokenized_text)

['[CLS]', '대한민국', '##의', '수도', '##는', '서울', '##입니다', '.', '[SEP]', '서울', '##의', '대표', '##적인', '관광지', '##는', '[MASK]', '##타', '##워', '입니다', '.', '[SEP]']


In [30]:
# 정수 인코딩
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

[2, 4892, 2079, 4438, 2259, 3671, 12190, 18, 3, 3671, 2079, 3661, 31221, 9417, 2259, 4, 2256, 2667, 3714, 18, 3]


In [31]:
# segment_ids 만들기
def get_segment_ids(tokenized_text):
    segment_ids = []
    current_segment_id = 0
    for token in tokenized_text:
        segment_ids.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id += 1
    return segment_ids


segment_ids = get_segment_ids(tokenized_text)
print(segment_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [32]:
# 파이토치 텐서 생성
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensor = torch.tensor([segment_ids])

print(tokens_tensor)
print(segments_tensor)

tensor([[    2,  4892,  2079,  4438,  2259,  3671, 12190,    18,     3,  3671,
          2079,  3661, 31221,  9417,  2259,     4,  2256,  2667,  3714,    18,
             3]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [33]:
# [MASK] 토큰으로 마스킹된 부분을 예측할 수 있는 모델 불러오기
model = BertForMaskedLM.from_pretrained(
    model_name
)  # 모델과 토크나이저의 모델명은 동일해야 한다.
model

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [34]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

tokens_tensor = tokens_tensor.to(device)
segments_tensor = segments_tensor.to(device)
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [35]:
# pred mode
model.eval()

with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensor)
    predictions = outputs[0]

predictions.shape

torch.Size([1, 21, 32000])

In [36]:
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_index

15555

In [37]:
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token

'롯데월드'

## 파이프라인을 이용한 예측
대부분의 자연어로 할 수 있는 처리들을 파이프라인화 하여 입력 부터 예측까지 하나의 단계로 완성할 수 있게 해 준다.

In [38]:
from transformers import pipeline

# 트랜스포머 모델을 이용해서 어떤 작업(task)을 수행할지 지정하고, 해당하는 모델을 넣어주면 된다.
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device="mps")

In [39]:
result = pipe(
    "부산광역시는 대표적인 한국의 도시입니다. 부산의 대표적인 관광지는 [MASK] 입니다."
)
result

[{'score': 0.5292269587516785,
  'token': 9568,
  'token_str': '해운대',
  'sequence': '부산광역시는 대표적인 한국의 도시입니다. 부산의 대표적인 관광지는 해운대 입니다.'},
 {'score': 0.0649060606956482,
  'token': 11615,
  'token_str': '해수욕장',
  'sequence': '부산광역시는 대표적인 한국의 도시입니다. 부산의 대표적인 관광지는 해수욕장 입니다.'},
 {'score': 0.04403737187385559,
  'token': 3902,
  'token_str': '부산',
  'sequence': '부산광역시는 대표적인 한국의 도시입니다. 부산의 대표적인 관광지는 부산 입니다.'},
 {'score': 0.04394268989562988,
  'token': 10509,
  'token_str': '온천',
  'sequence': '부산광역시는 대표적인 한국의 도시입니다. 부산의 대표적인 관광지는 온천 입니다.'},
 {'score': 0.024953261017799377,
  'token': 15879,
  'token_str': '케이블카',
  'sequence': '부산광역시는 대표적인 한국의 도시입니다. 부산의 대표적인 관광지는 케이블카 입니다.'}]

# RoBERTa를 이용한 텍스트 분류
- NLI

In [40]:
from transformers import pipeline, AutoTokenizer

classifier = pipeline(
    "text-classification", model="Huffon/klue-roberta-base-nli", top_k=None
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [41]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")

* ENTAILMENT : 논리적인 문장 구조
* NEUTRAL : 중립적인 문장 구조
* CONTRADICTION : 모순적인 문장 구조

In [42]:
classifier(
    f"나는 악기를 연주하는 것을 좋아한다. {tokenizer.sep_token} 나는 악기를 다루는 것이 싫다."
)

[[{'label': 'CONTRADICTION', 'score': 0.999235987663269},
  {'label': 'NEUTRAL', 'score': 0.0004199378308840096},
  {'label': 'ENTAILMENT', 'score': 0.0003440291038714349}]]